![image](../imgs/databites_header.png)

# Integrating DuckDB & Python: An Analytics Guide
**A Hands-On Guide Using Python**

DuckDB is a fast, in-process database for modern data analysis. Unlike traditional databases, it runs inside your Python app—no server needed—offering high-performance queries with columnar storage and vectorized execution, making it ideal for complex analytical workloads.

You’ll learn how to:

- Set up and query DuckDB directly in Python
- Load and analyze real-world datasets (CSV, Parquet, JSON)
- Apply core SQL techniques like filtering, sorting, and calculated columns

This notebook is ideal for beginners in data science or analysts wanting to enhance their time series skills using real-world datasets.

**Prerequisites:**
- Python 3.13 or later installed
- A basic understanding of SQL and data analysis in Python

# Preparing the Environment and Loading the Data
- Make sure you have Python 3.13+ and basic SQL/Python skills.
- Create a project directory, set up a virtual environment, and install DuckDB:


## Installing and importing the required libraries

In [27]:
!pip install duckdb


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## First steps with DuckDB
This script creates an in-memory DuckDB connection and runs a basic SQL query. The result is returned as a list of tuples, with each tuple representing a row from the query result.

In [28]:
import duckdb

# Create a connection to an in-memory database
conn = duckdb.connect()

# Execute a simple query and fetch the result
result = conn.execute("SELECT 'Hello, world!' AS message").fetchall()
print(result)

[('Hello, world!',)]


In this example, main.py reuses the conn object from analyzer.py to run a query. The result is fetched as a list of tuples. Separating the connection logic keeps your code clean and modular.

In [29]:
# Execute a simple query
result = conn.execute("SELECT 42 AS answer").fetchall()
print(result)

[(42,)]


Each query returns a list of tuples—DuckDB’s default format when using `.fetchall()`. This makes it easy to directly work with small datasets in Python without needing additional libraries.

In [30]:
import pandas as pd
source = "data/airline-passengers.csv"

df = pd.read_csv(f"{source}")
df.head()

,month,total_passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


## Working with external data sources in DuckDB
One of DuckDB’s standout features is its ability to query external data files directly—without needing to import them into a database or load entire datasets into memory. Unlike traditional databases that require data to be ingested first, DuckDB supports a "zero-copy" execution model, allowing it to read only the data required for a given query.

This approach brings several key advantages:
- **Minimal memory usage:** Only the relevant portions of the file are read into memory.
- **No import/export overhead:** Query your data in place—no need to move or duplicate it.
- **Streamlined workflows:** Easily query across multiple files and formats using a single SQL statement.

DuckDB supports a variety of file formats, especially columnar and analytics-friendly types like Parquet and CSV. In the next section, you'll learn how to connect to and query these external sources efficiently.


In [31]:
import duckdb

# Query data directly from a CSV file
result = duckdb.query(f"SELECT * FROM '{source}'").fetchall()
print(result)

[('1949-01', 112), ('1949-02', 118), ('1949-03', 132), ('1949-04', 129), ('1949-05', 121), ('1949-06', 135), ('1949-07', 148), ('1949-08', 148), ('1949-09', 136), ('1949-10', 119), ('1949-11', 104), ('1949-12', 118), ('1950-01', 115), ('1950-02', 126), ('1950-03', 141), ('1950-04', 135), ('1950-05', 125), ('1950-06', 149), ('1950-07', 170), ('1950-08', 170), ('1950-09', 158), ('1950-10', 133), ('1950-11', 114), ('1950-12', 140), ('1951-01', 145), ('1951-02', 150), ('1951-03', 178), ('1951-04', 163), ('1951-05', 172), ('1951-06', 178), ('1951-07', 199), ('1951-08', 199), ('1951-09', 184), ('1951-10', 162), ('1951-11', 146), ('1951-12', 166), ('1952-01', 171), ('1952-02', 180), ('1952-03', 193), ('1952-04', 181), ('1952-05', 183), ('1952-06', 218), ('1952-07', 230), ('1952-08', 242), ('1952-09', 209), ('1952-10', 191), ('1952-11', 172), ('1952-12', 194), ('1953-01', 196), ('1953-02', 196), ('1953-03', 236), ('1953-04', 235), ('1953-05', 229), ('1953-06', 243), ('1953-07', 264), ('1953-08

Let's build on our previous example with the data.csv file by starting with more useful queries. We'll progressively add more sophisticated SQL features to show what DuckDB can do.

Create a new file called explore_data.py and let's start with some practical operations:


### Filtering rows
When analyzing data, you often need to focus on specific subsets that meet certain criteria. DuckDB makes this easy with SQL's WHERE clause.
The WHERE clause acts like a filter, only returning rows that match your condition. You can use comparison operators like >, <, =, <=, >=, and <> (not equal), along with logical operators like AND, OR, and NOT to create more complex filters.

The `WHERE` clause acts like a filter, only returning rows that match your condition. You can use comparison operators like >, <, =, <=, >=, and <> (not equal), along with logical operators like AND, OR, and NOT to create more complex filters.

In [32]:
import duckdb

# Select only students with a score above 80
result = duckdb.query(f"SELECT * FROM '{source}' WHERE total_passengers > 500").fetchall()
result

[('1958-08', 505),
 ('1959-07', 548),
 ('1959-08', 559),
 ('1960-06', 535),
 ('1960-07', 622),
 ('1960-08', 606),
 ('1960-09', 508)]

### Sorting results
Data analysis often requires organizing results in a specific order. SQL's ORDER BY clause lets you sort your data based on one or more columns.

The ORDER BY clause sorts your results based on the specified column. By default, sorting is in ascending order (ASC), but you can use DESC for descending order as shown here. You can also sort by multiple columns by listing them separated by commas, like ORDER BY column1, column2 DESC.

In [33]:
import duckdb

# Select only students with a score above 80
result = duckdb.query(f"SELECT * FROM '{source}' WHERE total_passengers > 500").fetchall()

print("Months with a higher total amount of passengers greater than 500")
print(result)

#Sort months by number of passengers
sorted_result = duckdb.query(f"SELECT * FROM '{source}' ORDER BY total_passengers DESC ").fetchall()
print("\Months sorted by total traffic:")
print(sorted_result)


Months with a higher total amount of passengers greater than 500
[('1958-08', 505), ('1959-07', 548), ('1959-08', 559), ('1960-06', 535), ('1960-07', 622), ('1960-08', 606), ('1960-09', 508)]
\Months sorted by total traffic:
[('1960-07', 622), ('1960-08', 606), ('1959-08', 559), ('1959-07', 548), ('1960-06', 535), ('1960-09', 508), ('1958-08', 505), ('1958-07', 491), ('1959-06', 472), ('1960-05', 472), ('1957-08', 467), ('1957-07', 465), ('1959-09', 463), ('1960-04', 461), ('1960-10', 461), ('1958-06', 435), ('1960-12', 432), ('1957-06', 422), ('1959-05', 420), ('1960-03', 419), ('1960-01', 417), ('1956-07', 413), ('1959-10', 407), ('1959-03', 406), ('1956-08', 405), ('1959-12', 405), ('1957-09', 404), ('1958-09', 404), ('1959-04', 396), ('1960-02', 391), ('1960-11', 390), ('1956-06', 374), ('1955-07', 364), ('1958-05', 363), ('1958-03', 362), ('1959-11', 362), ('1959-01', 360), ('1958-10', 359), ('1957-03', 356), ('1956-09', 355), ('1957-05', 355), ('1957-04', 348), ('1958-04', 348), 

### Adding calculated columns
SQL makes it easy to create new columns using expressions. This lets you transform your data and generate new insights directly in your query.

The AS keyword lets you name the result of an expression, creating a new column in your output. You can use arithmetic operators like +, -, *, and /, as well as many built-in functions. These calculated columns exist only in your query results and don't modify the original data file.

In [34]:
import duckdb

...

# Add 10 bonus points to each score
bonus_result = duckdb.query(f"""
    SELECT
        month,
        total_passengers,
        total_passengers/1000 AS traffic_in_thousands
    FROM '{source}'
""").fetchall()
print("\nScores with 10 bonus points:")
print(bonus_result)



Scores with 10 bonus points:
[('1949-01', 112, 0.112), ('1949-02', 118, 0.118), ('1949-03', 132, 0.132), ('1949-04', 129, 0.129), ('1949-05', 121, 0.121), ('1949-06', 135, 0.135), ('1949-07', 148, 0.148), ('1949-08', 148, 0.148), ('1949-09', 136, 0.136), ('1949-10', 119, 0.119), ('1949-11', 104, 0.104), ('1949-12', 118, 0.118), ('1950-01', 115, 0.115), ('1950-02', 126, 0.126), ('1950-03', 141, 0.141), ('1950-04', 135, 0.135), ('1950-05', 125, 0.125), ('1950-06', 149, 0.149), ('1950-07', 170, 0.17), ('1950-08', 170, 0.17), ('1950-09', 158, 0.158), ('1950-10', 133, 0.133), ('1950-11', 114, 0.114), ('1950-12', 140, 0.14), ('1951-01', 145, 0.145), ('1951-02', 150, 0.15), ('1951-03', 178, 0.178), ('1951-04', 163, 0.163), ('1951-05', 172, 0.172), ('1951-06', 178, 0.178), ('1951-07', 199, 0.199), ('1951-08', 199, 0.199), ('1951-09', 184, 0.184), ('1951-10', 162, 0.162), ('1951-11', 146, 0.146), ('1951-12', 166, 0.166), ('1952-01', 171, 0.171), ('1952-02', 180, 0.18), ('1952-03', 193, 0.193),

### Using CASE expressions

For more complex transformations, SQL provides the CASE expression. This works similarly to if-else statements in programming languages, allowing you to apply conditional logic in your queries.


In [35]:
import duckdb

# Previous code...

# Assign grades based on score

segmented_result = duckdb.query(f"""
    SELECT
        month,
        total_passengers,
        CASE
            WHEN total_passengers >= 100 THEN 'HIGH'
            WHEN total_passengers >= 50 THEN 'MEDIUM'
            ELSE 'LOW'
        END AS affluency
    FROM '{source}'
""").fetchall()

print("\nStudents with letter grades:")
print(segmented_result)



Students with letter grades:
[('1949-01', 112, 'HIGH'), ('1949-02', 118, 'HIGH'), ('1949-03', 132, 'HIGH'), ('1949-04', 129, 'HIGH'), ('1949-05', 121, 'HIGH'), ('1949-06', 135, 'HIGH'), ('1949-07', 148, 'HIGH'), ('1949-08', 148, 'HIGH'), ('1949-09', 136, 'HIGH'), ('1949-10', 119, 'HIGH'), ('1949-11', 104, 'HIGH'), ('1949-12', 118, 'HIGH'), ('1950-01', 115, 'HIGH'), ('1950-02', 126, 'HIGH'), ('1950-03', 141, 'HIGH'), ('1950-04', 135, 'HIGH'), ('1950-05', 125, 'HIGH'), ('1950-06', 149, 'HIGH'), ('1950-07', 170, 'HIGH'), ('1950-08', 170, 'HIGH'), ('1950-09', 158, 'HIGH'), ('1950-10', 133, 'HIGH'), ('1950-11', 114, 'HIGH'), ('1950-12', 140, 'HIGH'), ('1951-01', 145, 'HIGH'), ('1951-02', 150, 'HIGH'), ('1951-03', 178, 'HIGH'), ('1951-04', 163, 'HIGH'), ('1951-05', 172, 'HIGH'), ('1951-06', 178, 'HIGH'), ('1951-07', 199, 'HIGH'), ('1951-08', 199, 'HIGH'), ('1951-09', 184, 'HIGH'), ('1951-10', 162, 'HIGH'), ('1951-11', 146, 'HIGH'), ('1951-12', 166, 'HIGH'), ('1952-01', 171, 'HIGH'), ('1952-